In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from scipy.stats.stats import ttest_ind
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

usEducation = pd.read_sql_query('select * from useducation',con=engine)
engine.dispose()

usEducation.columns = usEducation.columns.str.lower()


In [2]:
usEducation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
primary_key                     1492 non-null object
state                           1492 non-null object
year                            1492 non-null int64
enroll                          1229 non-null float64
total_revenue                   1280 non-null float64
federal_revenue                 1280 non-null float64
state_revenue                   1280 non-null float64
local_revenue                   1280 non-null float64
total_expenditure               1280 non-null float64
instruction_expenditure         1280 non-null float64
support_services_expenditure    1280 non-null float64
other_expenditure               1229 non-null float64
capital_outlay_expenditure      1280 non-null float64
grades_pk_g                     1319 non-null float64
grades_kg_g                     1360 non-null float64
grades_4_g                      1361 non-null float64
grades_8_g                      1

In [3]:
fill_list = ["enroll", "total_revenue", "federal_revenue", 
             "state_revenue", "federal_revenue", "total_expenditure", 
             "instruction_expenditure", "support_services_expenditure",
             "other_expenditure", "capital_outlay_expenditure", "grades_pk_g", 
             "grades_kg_g", "grades_4_g", "grades_8_g", "grades_12_g", "grades_all_g", 
             "avg_math_4_score", "avg_math_8_score",'avg_reading_4_score','avg_reading_4_score','avg_reading_8_score']
for col in fill_list:
    usEducation.loc[:, col] = usEducation.loc[:, col].fillna(usEducation.loc[:, col].mean())

In [4]:
usEducation.isnull().sum()/usEducation.isnull().count()

primary_key                     0.000000
state                           0.000000
year                            0.000000
enroll                          0.000000
total_revenue                   0.000000
federal_revenue                 0.000000
state_revenue                   0.000000
local_revenue                   0.142091
total_expenditure               0.000000
instruction_expenditure         0.000000
support_services_expenditure    0.000000
other_expenditure               0.000000
capital_outlay_expenditure      0.000000
grades_pk_g                     0.000000
grades_kg_g                     0.000000
grades_4_g                      0.000000
grades_8_g                      0.000000
grades_12_g                     0.000000
grades_1_8_g                    0.087802
grades_9_12_g                   0.087802
grades_all_g                    0.000000
avg_math_4_score                0.000000
avg_math_8_score                0.000000
avg_reading_4_score             0.000000
avg_reading_8_sc

# 1) Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [5]:
usEducation["scoreVariable"] = (usEducation["grades_4_g"]*((usEducation["avg_math_4_score"] + usEducation["avg_reading_4_score"])*0.5) + usEducation["grades_8_g"]
                                 * ((usEducation["avg_math_8_score"] + usEducation["avg_reading_8_score"])*0.5))/(usEducation["grades_4_g"] + usEducation["grades_8_g"])


# 2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [6]:
usEducation[["scoreVariable", "total_expenditure", "instruction_expenditure",
              "support_services_expenditure", "other_expenditure", "capital_outlay_expenditure"]].corr()

,scoreVariable,total_expenditure,instruction_expenditure,support_services_expenditure,other_expenditure,capital_outlay_expenditure
scoreVariable,1.000000,0.069725,0.075448,0.072963,0.010777,0.025506
total_expenditure,0.069725,1.000000,0.991347,0.993314,0.939463,0.932447
instruction_expenditure,0.075448,0.991347,1.000000,0.976889,0.902964,0.896103
support_services_expenditure,0.072963,0.993314,0.976889,1.000000,0.945724,0.917782
other_expenditure,0.010777,0.939463,0.902964,0.945724,1.000000,0.916854
capital_outlay_expenditure,0.025506,0.932447,0.896103,0.917782,0.916854,1.000000


The instruction_expenditure is correlated to the scoreVariable created.

# 3) Now, apply PCA to the four expenditure types. How much of the total variance is explained by the first component?¶

In [7]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = usEducation[["instruction_expenditure", "support_services_expenditure",
                  "other_expenditure", "capital_outlay_expenditure"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
usEducation["pca"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94464539]


From the PCA, 94% is the total variance explained in the dataset.

# 4) What is the correlation between the overall score variable and the first principal component?¶

In [8]:
usEducation[["scoreVariable","pca", "total_expenditure", "instruction_expenditure",
              "support_services_expenditure", "other_expenditure", "capital_outlay_expenditure"]].corr()

,scoreVariable,pca,total_expenditure,instruction_expenditure,support_services_expenditure,other_expenditure,capital_outlay_expenditure
scoreVariable,1.000000,0.047716,0.069725,0.075448,0.072963,0.010777,0.025506
pca,0.047716,1.000000,0.992182,0.971529,0.988095,0.968572,0.959297
total_expenditure,0.069725,0.992182,1.000000,0.991347,0.993314,0.939463,0.932447
instruction_expenditure,0.075448,0.971529,0.991347,1.000000,0.976889,0.902964,0.896103
support_services_expenditure,0.072963,0.988095,0.993314,0.976889,1.000000,0.945724,0.917782
other_expenditure,0.010777,0.968572,0.939463,0.902964,0.945724,1.000000,0.916854
capital_outlay_expenditure,0.025506,0.959297,0.932447,0.896103,0.917782,0.916854,1.000000


The first pricipal component has higher values than the overall score. 

# 5) If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?¶

I would choose expenditure variable because all of its numbers are closer to one. My second choice would be Support service expenditure, its more correlated. 